In [2]:
!pip install tensorflow autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 70.8 MB/s eta 0:00:00


In [3]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('drive/MyDrive/datasets/cancer_dataset.csv')

In [5]:
df = data.copy()
df.head(2)

,id_ответа,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,...,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий
0,1,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,до 42 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0
1,2,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,от 43 до 60 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0


In [6]:
df['Комментарий'] = df['Комментарий'].replace('0', 'Отсутствует')
df['Альтернатива, если 50% - обязательно'] = df['Альтернатива, если 50% - обязательно'].replace('0', 'Отсутствует')

In [7]:
# Удаление ненужных столбцов
df.drop(columns=['id_ответа'], inplace=True)

In [8]:
for col in df.columns:
    print(f'Уникальные значения столбца {col}: {df[col].unique()}')

Уникальные значения столбца Раса: ['Азиатская' 'Европейская' 'Другая']
Уникальные значения столбца Пол: [' Мужской' ' Женский']
Уникальные значения столбца Возраст: [' До 70' ' >70']
Уникальные значения столбца Статус курения: [' В настоящее время'
 ' Курение в прошлом (бросил более 1 месяца до 1 года)'
 ' Курение в прошлом (бросил более 1 года)']
Уникальные значения столбца ECOG: [' 0-1' '2']
Уникальные значения столбца Есть опухолевая нагрузка? (симптомная опухоль): [' да' ' нет']
Уникальные значения столбца Ко-мутации KRAS: [' да' ' нет']
Уникальные значения столбца Ко-мутации p53.: [' да' ' нет']
Уникальные значения столбца Ко-мутации STK11: [' да' ' нет']
Уникальные значения столбца Ко-мутации KEAP1: [' да' ' нет']
Уникальные значения столбца Срок от окончания ХЛТ: ['до 42 дней' 'от 43 до 60 дней' 'более 61 дня']
Уникальные значения столбца Молекулярный статус (только для неплоскоклеточного рака): [' нет мутаций' ' не исследовались' ' EGFR редкий вариант' ' EGFR ex19'
 ' EGFR ex21

In [9]:
# Создание нового столбца "Уверенность"
df['Уверенность'] = 'Нет уверенности'

# Заполнение столбца "Уверенность" в зависимости от значений в трех указанных столбцах
df.loc[df['Поставьте галочку, если уверены на 100%'], 'Уверенность'] = '100%'
df.loc[df['Если 75%'], 'Уверенность'] = '75%'
df.loc[df['Если 50%'], 'Уверенность'] = '50%'

df.drop(columns=['Поставьте галочку, если уверены на 100%', 'Если 75%', 'Если 50%'], inplace=True)
df.head(2)

,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,Ко-мутации KEAP1,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Альтернатива, если 50% - обязательно",Комментарий,Уверенность
0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,до 42 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,Отсутствует,Отсутствует,100%
1,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,от 43 до 60 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,Отсутствует,Отсутствует,100%


In [10]:
# Проход по каждому столбцу и удаление пробелов в начале строк (если столбец строковый)
for column in df.columns:
    if df[column].dtype == 'O':
        df[column] = df[column].str.strip()

In [11]:
# Проверка уникальных значений каждого столбца
for col in df.columns:
    print(f'Уникальные значения столбца {col}: {df[col].unique()}')

Уникальные значения столбца Раса: ['Азиатская' 'Европейская' 'Другая']
Уникальные значения столбца Пол: ['Мужской' 'Женский']
Уникальные значения столбца Возраст: ['До 70' '>70']
Уникальные значения столбца Статус курения: ['В настоящее время' 'Курение в прошлом (бросил более 1 месяца до 1 года)'
 'Курение в прошлом (бросил более 1 года)']
Уникальные значения столбца ECOG: ['0-1' '2']
Уникальные значения столбца Есть опухолевая нагрузка? (симптомная опухоль): ['да' 'нет']
Уникальные значения столбца Ко-мутации KRAS: ['да' 'нет']
Уникальные значения столбца Ко-мутации p53.: ['да' 'нет']
Уникальные значения столбца Ко-мутации STK11: ['да' 'нет']
Уникальные значения столбца Ко-мутации KEAP1: ['да' 'нет']
Уникальные значения столбца Срок от окончания ХЛТ: ['до 42 дней' 'от 43 до 60 дней' 'более 61 дня']
Уникальные значения столбца Молекулярный статус (только для неплоскоклеточного рака): ['нет мутаций' 'не исследовались' 'EGFR редкий вариант' 'EGFR ex19'
 'EGFR ex21' 'ALK позитивный']
Уник

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Предобработка данных
le = LabelEncoder()
target_columns = ['Ответ эксперта (Лактионов)', 'Уверенность', 'Комментарий', 'Альтернатива, если 50% - обязательно']

for column in target_columns:
    df[column] = le.fit_transform(df[column])

# Разделение данных на тренировочный и тестовый наборы
X = df.drop(['Ответ эксперта (Лактионов)', 'Уверенность', 'Комментарий', 'Альтернатива, если 50% - обязательно'], axis=1)
y_expert = df['Ответ эксперта (Лактионов)']
y_confidence = df['Уверенность']
y_comment = df['Комментарий']
y_alternative = df['Альтернатива, если 50% - обязательно']

X_train, X_test, y_expert_train, y_expert_test, y_confidence_train, y_confidence_test, y_comment_train, y_comment_test, y_alternative_train, y_alternative_test = train_test_split(
    X, y_expert, y_confidence, y_comment, y_alternative, test_size=0.2, random_state=42
)

# Преобразование категориальных признаков
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)


## Model 1

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

input_layer = Input(shape=(X_train.shape[1],))

output_expert = Dense(len(df['Ответ эксперта (Лактионов)'].unique()), activation='softmax', name='expert')(input_layer)
output_confidence = Dense(len(df['Уверенность'].unique()), activation='softmax', name='confidence')(input_layer)
output_comment = Dense(len(df['Комментарий'].unique()), activation='softmax', name='comment')(input_layer)
output_alternative = Dense(len(df['Альтернатива, если 50% - обязательно'].unique()), activation='softmax', name='alternative')(input_layer)

model = Model(inputs=input_layer, outputs=[output_expert, output_confidence, output_comment, output_alternative])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, [y_expert_train, y_confidence_train, y_comment_train, y_alternative_train], epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
2489/2489 [==============================] - 11s 4ms/step - loss: 3.7368 - expert_loss: 0.6942 - confidence_loss: 0.8645 - comment_loss: 1.2304 - alternative_loss: 0.9477 - expert_accuracy: 0.7591 - confidence_accuracy: 0.6281 - comment_accuracy: 0.6709 - alternative_accuracy: 0.6067 - val_loss: 2.5607 - val_expert_loss: 0.3909 - val_confidence_loss: 0.7108 - val_comment_loss: 0.7163 - val_alternative_loss: 0.7427 - val_expert_accuracy: 0.9300 - val_confidence_accuracy: 0.7347 - val_comment_accuracy: 0.8646 - val_alternative_accuracy: 0.6707
Epoch 2/10
2489/2489 [==============================] - 7s 3ms/step - loss: 2.1967 - expert_loss: 0.2864 - confidence_loss: 0.6770 - comment_loss: 0.5519 - alternative_loss: 0.6815 - expert_accuracy: 0.9637 - confidence_accuracy: 0.7469 - comment_accuracy: 0.8794 - alternative_accuracy: 0.6787 - val_loss: 1.9587 - val_expert_loss: 0.2135 - val_confidence_loss: 0.6605 - val_comment_loss: 0.4357 - val_alternative_loss: 0.6489 - val_expert_

In [14]:
# Оценка модели на тестовом наборе
results = model.evaluate(X_test, [y_expert_test, y_confidence_test, y_comment_test, y_alternative_test], batch_size=64)

print("Test Accuracy - Expert:", results[4])
print("Test Accuracy - Confidence:", results[5])
print("Test Accuracy - Comment:", results[6])
print("Test Accuracy - Alternative:", results[7])

778/778 [==============================] - 2s 2ms/step - loss: 1.3677 - expert_loss: 0.0177 - confidence_loss: 0.6451 - comment_loss: 0.1267 - alternative_loss: 0.5782 - expert_accuracy: 1.0000 - confidence_accuracy: 0.7473 - comment_accuracy: 0.9478 - alternative_accuracy: 0.6722
Test Accuracy - Expert: 0.5782084465026855
Test Accuracy - Confidence: 1.0
Test Accuracy - Comment: 0.7473024129867554
Test Accuracy - Alternative: 0.9478369355201721


## Model 2

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

input_layer = Input(shape=(X_train.shape[1],))

# Скрытые слои и слои Dropout для предотвращения переобучения
hidden_layer1 = Dense(128, activation='relu')(input_layer)
dropout1 = Dropout(0.5)(hidden_layer1)
hidden_layer2 = Dense(64, activation='relu')(dropout1)
dropout2 = Dropout(0.5)(hidden_layer2)

output_expert = Dense(len(df['Ответ эксперта (Лактионов)'].unique()), activation='softmax', name='expert')(dropout2)
output_confidence = Dense(len(df['Уверенность'].unique()), activation='softmax', name='confidence')(dropout2)
output_comment = Dense(len(df['Комментарий'].unique()), activation='softmax', name='comment')(dropout2)
output_alternative = Dense(len(df['Альтернатива, если 50% - обязательно'].unique()), activation='softmax', name='alternative')(dropout2)

model = Model(inputs=input_layer, outputs=[output_expert, output_confidence, output_comment, output_alternative])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, [y_expert_train, y_confidence_train, y_comment_train, y_alternative_train], epochs=5, batch_size=128, validation_split=0.2)

results = model.evaluate(X_test, [y_expert_test, y_confidence_test, y_comment_test, y_alternative_test], batch_size=128)

print("Test Accuracy - Expert:", results[4])
print("Test Accuracy - Confidence:", results[5])
print("Test Accuracy - Comment:", results[6])
print("Test Accuracy - Alternative:", results[7])

Epoch 1/5
1245/1245 [==============================] - 7s 4ms/step - loss: 2.1322 - expert_loss: 0.3175 - confidence_loss: 0.5253 - comment_loss: 0.7342 - alternative_loss: 0.5553 - expert_accuracy: 0.8772 - confidence_accuracy: 0.7728 - comment_accuracy: 0.7507 - alternative_accuracy: 0.7654 - val_loss: 0.4058 - val_expert_loss: 0.0209 - val_confidence_loss: 0.1641 - val_comment_loss: 0.1067 - val_alternative_loss: 0.1141 - val_expert_accuracy: 0.9902 - val_confidence_accuracy: 0.9509 - val_comment_accuracy: 0.9769 - val_alternative_accuracy: 0.9751
Epoch 2/5
1245/1245 [==============================] - 7s 5ms/step - loss: 0.6524 - expert_loss: 0.0647 - confidence_loss: 0.2115 - comment_loss: 0.1986 - alternative_loss: 0.1776 - expert_accuracy: 0.9784 - confidence_accuracy: 0.9204 - comment_accuracy: 0.9349 - alternative_accuracy: 0.9335 - val_loss: 0.0975 - val_expert_loss: 0.0032 - val_confidence_loss: 0.0421 - val_comment_loss: 0.0199 - val_alternative_loss: 0.0323 - val_expert_acc

## Model 3

In [18]:
input_layer = Input(shape=(X_train.shape[1],))

hidden_layer1 = Dense(64, activation='relu')(input_layer)
dropout1 = Dropout(0.3)(hidden_layer1)
hidden_layer2 = Dense(32, activation='relu')(dropout1)
dropout2 = Dropout(0.3)(hidden_layer2)

output_expert = Dense(len(df['Ответ эксперта (Лактионов)'].unique()), activation='softmax', name='expert')(dropout2)
output_confidence = Dense(len(df['Уверенность'].unique()), activation='softmax', name='confidence')(dropout2)
output_comment = Dense(len(df['Комментарий'].unique()), activation='softmax', name='comment')(dropout2)
output_alternative = Dense(len(df['Альтернатива, если 50% - обязательно'].unique()), activation='softmax', name='alternative')(dropout2)

model = Model(inputs=input_layer, outputs=[output_expert, output_confidence, output_comment, output_alternative])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, [y_expert_train, y_confidence_train, y_comment_train, y_alternative_train], epochs=5, batch_size=128, validation_split=0.2)

results = model.evaluate(X_test, [y_expert_test, y_confidence_test, y_comment_test, y_alternative_test], batch_size=128)

print("Test Accuracy - Expert:", results[4])
print("Test Accuracy - Confidence:", results[5])
print("Test Accuracy - Comment:", results[6])
print("Test Accuracy - Alternative:", results[7])


Epoch 1/5
1245/1245 [==============================] - 7s 4ms/step - loss: 2.3630 - expert_loss: 0.3586 - confidence_loss: 0.5652 - comment_loss: 0.8271 - alternative_loss: 0.6121 - expert_accuracy: 0.8619 - confidence_accuracy: 0.7505 - comment_accuracy: 0.7145 - alternative_accuracy: 0.7422 - val_loss: 0.5615 - val_expert_loss: 0.0339 - val_confidence_loss: 0.1975 - val_comment_loss: 0.1570 - val_alternative_loss: 0.1731 - val_expert_accuracy: 0.9901 - val_confidence_accuracy: 0.9542 - val_comment_accuracy: 0.9492 - val_alternative_accuracy: 0.9600
Epoch 2/5
1245/1245 [==============================] - 4s 3ms/step - loss: 0.8070 - expert_loss: 0.0797 - confidence_loss: 0.2470 - comment_loss: 0.2515 - alternative_loss: 0.2289 - expert_accuracy: 0.9722 - confidence_accuracy: 0.9067 - comment_accuracy: 0.9131 - alternative_accuracy: 0.9084 - val_loss: 0.1957 - val_expert_loss: 0.0074 - val_confidence_loss: 0.0692 - val_comment_loss: 0.0558 - val_alternative_loss: 0.0633 - val_expert_acc